In [1]:
import wandb
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'model_zoo.ipynb'

In [2]:
wandb.login()

True

# first initial proposition

In [3]:
from classification_models_1D.tfkeras import Classifiers

In [4]:
from termcolor import colored

def step_color(bloc_order, position):
    formattedText = []
    for pos, i in enumerate(bloc_order):
        pos += 1
        if pos != position and pos != len(bloc_order):
            formattedText.append('{} -> '.format(i))
        elif pos == position:
            formattedText.append(colored('{}'.format(i),'white','on_red'))
            formattedText.append(' -> ')

        else:
            formattedText.append(i)

    return ''.join(formattedText)


In [5]:
def init_wandb(dict_yaml, step='model'):
    
    if step == 'model':
        config={
            'name': dict_yaml['name_sweep'],
            'method': dict_yaml['method'],
            'metric': {'goal': dict_yaml['metric']['goal'], 'name': dict_yaml['metric']['name']},
            'parameters':
            {
                'models': {'values':dict_yaml['config']['models']}

            }
        }    

    return config

In [6]:
from keras import backend as K
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score

def macro_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred, multi_class='ovr',average='macro')

def micro_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred, multi_class='ovr', average='micro')

def sample_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred,  multi_class='ovr' , average='samples')

def averaged_metric(y_true, y_pred):
    return (micro_auc(y_true, y_pred) + macro_auc(y_true, y_pred))/2


def auc_pr(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    return roc_auc_score(recall, precision)



In [7]:
import pprint
pp = pprint.PrettyPrinter(depth=4)

from classification_models_1D.tfkeras import Classifiers
from termcolor import colored
import yaml
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import numpy as np
import os
from statistics import mean, stdev 
import pickle

data_dir = '/volume/core_model/data_split/'

with open('test.yml', 'r') as file:
    prime_service = yaml.safe_load(file)
prime_service

#print user paraeters

print(colored('Curently running core model optimization', 'green'))
print(colored('Job name :', 'green'), colored(prime_service['run_name'], 'cyan'))
print(colored('Sweep name :', 'green'), colored(prime_service['name_sweep'], 'cyan'))
print(colored('Optimization method :', 'green'), colored(prime_service['method'], 'cyan'))
print(colored('Optimization applied to the metric :', 'green'), colored(prime_service['metric']['name'], 'cyan'))
print(colored('Optimization goal :', 'green'), colored(prime_service['metric']['goal'], 'cyan'))
print('\n')
print(colored('Chosen parameters are:', 'green'))
pp.pprint(prime_service)

block_counter = 1
print('\n')
print(colored('Currently running:','green'))
print(step_color(prime_service['bloc_order'], block_counter))
print('\n')
print(colored('Step {}.1:'.format(block_counter),'green'))
print(colored('Main backbone'))

# select a model to run
sweep_configuration = init_wandb(prime_service, step='model')
print(sweep_configuration)
sweep_id = wandb.sweep(sweep=sweep_configuration, project=prime_service['run_name'])


# first 1.1 block
#wandb.run=None

def block_1():
    print("adadadw")
    run = wandb.init()
    print("oooo")

    save_dict = {}

    #load model
    print(wandb.config)
    loaded_architecture, preprocess_input = Classifiers.get(wandb.config.models)

    print('Currently running {}'.format(wandb.config.models))

    model_selected = loaded_architecture(
    input_shape=(2500, 12),
    weights=None
    )

    optimizer_ = tf.keras.optimizers.Adam(learning_rate=0.01)
    loss_func = 'binary_crossentropy'

    model_selected.compile(optimizer=optimizer_, loss=loss_func, metrics=['accuracy',macro_auc,micro_auc,sample_auc,averaged_metric])     

    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=2)
    lrplateau = ReduceLROnPlateau(  
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=0,
    mode="auto", 
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-12)

    #load data

    list_scores = list()
    for split in range(3):

        curr_data_dir = os.path.join(data_dir,'split_{}'.format(split))
        X_train = np.load(os.path.join(curr_data_dir,'train_X.npy')) 
        Y_train = np.load(os.path.join(curr_data_dir,'train_Y.npy')) 

        X_val = np.load(os.path.join(curr_data_dir,'val_X.npy')) 
        Y_val = np.load(os.path.join(curr_data_dir,'val_Y.npy')) 

        history = model_selected.fit(X_train, Y_train, verbose=1,batch_size=64, epochs=250, callbacks=[early_stop,lrplateau], validation_data=(X_val,Y_val)) 

        #evaluate model
        X_test = np.load(os.path.join(curr_data_dir,'val_X.npy')) 
        Y_test = np.load(os.path.join(curr_data_dir,'val_Y.npy')) 
        Y_test_pred = model_selected.predict(X_test)

        dict_results = dict()
        for i in [macro_auc,micro_auc,sample_auc,averaged_metric,auc_pr]:
            dict_results.update({i.__name__:i(Y_test,Y_test_pred)})

        list_scores.append(averaged_metric(Y_test,Y_test_pred))

        if prime_service['save_info'] == True:
            save_dict.update({'{}_{}'.format(wandb.config.models,split):{'history':history.history, 'test_performances':dict_results}})

    print('Perfroances for {} were {} ± {}'.format(wandb.config.models,mean(list_scores), stdev(list_scores)))


    wandb.log({
    'model': wandb.config.models, 
    'macro_auc': dict_results['macro_auc'],
    'micro_auc': dict_results['micro_auc'],
    'sample_auc': dict_results['sample_auc'],
    'averaged_metric': dict_results['averaged_metric'],
    'auc_pr':dict_results['auc_pr']
    })

    if prime_service['bloc_order'] == True:
        with open(os.path.join(prime_service['save_dir'],'saved_outputs_detailed.{}.pickle'.format(prime_service['name_sweep'])), 'wb') as handle:
            pickle.dump(save_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


Curently running core model optimization
Job name : train run 1
Sweep name : sweep 1
Optimization method : bayes
Optimization applied to the metric : averaged_metric
Optimization goal : maximize


Chosen parameters are:
{'allow_ensemble': True,
 'best_to_select': 3,
 'bloc_order': ['model', 'optimizer', 'loss', 'regularization', 'fine_tuning'],
 'block_to_run': 1,
 'config': {'kernel_size': ['default', 5, 7, 9],
            'models': ['resnet18',
                       'resnet34',
                       'resnet50',
                       'resnet101',
                       'seresnet18',
                       'seresnet34',
                       'seresnet50',
                       'seresnet101',
                       'seresnext50',
                       'seresnext101',
                       'resnext50',
                       'resnext101',
                       'densenet121',
                       'densenet169',
                       'mobilenetv2',
                       'incept

In [8]:
wandb.agent(sweep_id, function=main, count=1)

wandb: Agent Starting Run: cnh7e2dd with config:
	models: EfficientNetV2B0
wandb: Agent Started Run: cnh7e2dd
adadadw


wandb: Wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


oooo
wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.9.7
    framework: keras
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.8.5
models:
  desc: null
  value: EfficientNetV2B0

Currently running EfficientNetV2B0


Process Process-1:
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/wandb/wandb_agent.py", line 64, in _start
    function()
  File "<ipython-input-7-67fb3e3d3006>", line 89, in main
    X_train = np.load(os.path.join(curr_data_dir,'train_X.npy'))
  File "/root/miniconda3/lib/python3.8/site-packages/numpy/lib/npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: '/volume/core_model/data_split/split_0/train_X.npy'


In [ ]:
Exception('problem')

Exception('problem')